#Hate Speech Detection (BERT) and Topic Modelling (BERTopic)

Import relevant libraries

In [2]:
"""Author: Melwyn D Souza, Reg No: R00209495"""
!pip install tensorflow-text
import pandas as pd
from sklearn.model_selection import train_test_split

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100

'Author: Melwyn D Souza, Reg No: R00209495'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.4 MB/s eta 0:00:00


Data Pre-Processing

The data is collected from 5 differnent online sources, the data is all the tweets from differnt users all over the world, this data is not configured (some files have multiilabelled data, some are labeled with sentiments etc) the way I want it for my HSD model, so the data has to be firstly made into two columns
1.   Tweets (string format)
2.   Lable - Hate/Non-Hate

Data is manually prepared by mergining data from 5 sources and contains almost balanced dataset
1.   #Total instances: 76679
2.   #Hate tweets: 30147
3.   #Non-Hate tweets: 46532

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/")
!ls

Mounted at /content/gdrive/
gdrive	sample_data


The data is imbalanced and we will sample only 30k tweets randomly out of both labels Hate & Non-Hate

We will call use two seperate dataframes to sample (calling them df_hs & df_nhs) then merge them both 

In [4]:
df= pd.read_csv("gdrive/My Drive/Thesis/dataset/data.csv", names=["tweet", "label"])
# df.head()
df.groupby("label").describe()
print(" - - "*20)
print(f"Imbalanced data:\n{df['label'].value_counts()}")
print(" - - "*20)

df_hs = df[df['label']==1]
df_nhs = df[df['label']==0]

df_hs = df_hs.sample(30000)
df_nhs = df_nhs.sample(30000)

df_balanced  = pd.concat([df_hs,df_nhs])

df_balanced['label'].value_counts()
df_balanced.head()
df_balanced.tail()

tweet                                                               
       count unique                                                top freq
label                                                                      
0      46532  44225  #model   i love u take with u all the time in ...  319
1      30147  28303  @user you might be a libtard if... #libtard  #...   40

 - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - - 
Imbalanced data:
0    46532
1    30147
Name: label, dtype: int64
 - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - - 


1    30000
0    30000
Name: label, dtype: int64

,tweet,label
75221,RT @ExposeFalsehood: @AbuAlbaraaSham above all...,1
62093,if I ever fall alseep with them on I wake up ...,1
47418,RT @Heissarcastic: If my jokes offend you:\n\n...,1
54922,Woke up feeling like a bitch. Gonna stop actin...,1
43905,It's funny when u get a black female mad they ...,1


,tweet,label
4046,i'm so happy! ð #happinesseverywhere #happ...,0
67807,@sschinke @georgieonthego she's claiming I thr...,0
74359,RT @RudawEnglish: #ISIS has begun retreating t...,0
13559,be n #healthy discipline is the bridge betwe...,0
5580,@user to all of us whom have immigrated from ...,0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['tweet'],df_balanced['label'], stratify=df_balanced['label'])
# X_train.head()  

In [11]:
bert_pre_process = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [9]:
def get_txt_embeddings(txt):
  pre_processed_txt = bert_pre_process(txt)
  return bert_encoder(pre_processed_txt)

The BERT models return a map with 3 important keys: pooled_output, sequence_output, encoder_outputs:

pooled_output represents each input sequence as a whole. The shape is [batch_size, H]. You can think of this as an embedding for the entire movie review.

sequence_output represents each input token in the context. The shape is [batch_size, seq_length, H]. You can think of this as a contextual embedding for every token in the movie review.

encoder_outputs are the intermediate activations of the L Transformer blocks. outputs["encoder_outputs"][i] is a Tensor of shape [batch_size, seq_length, 1024] with the outputs of the i-th Transformer block, for 0 <= i < L. The last value of the list is equal to sequence_output

In [13]:
get_txt_embeddings([
    "melwyn is a bright engineer but is lacking opportunities, or maybe trying many things at once",
    "hi this is mad",
    "wtf"
])

{'pooled_output': <tf.Tensor: shape=(3, 768), dtype=float32, numpy=
 array([[-0.47756723, -0.38555732, -0.8474778 , ..., -0.41609833,
         -0.49171314,  0.65542185],
        [-0.8913082 , -0.2525355 ,  0.17353486, ...,  0.21556185,
         -0.57174665,  0.9214097 ],
        [-0.872528  , -0.32431608, -0.4388646 , ..., -0.08474077,
         -0.6558136 ,  0.8447895 ]], dtype=float32)>,
 'default': <tf.Tensor: shape=(3, 768), dtype=float32, numpy=
 array([[-0.47756723, -0.38555732, -0.8474778 , ..., -0.41609833,
         -0.49171314,  0.65542185],
        [-0.8913082 , -0.2525355 ,  0.17353486, ...,  0.21556185,
         -0.57174665,  0.9214097 ],
        [-0.872528  , -0.32431608, -0.4388646 , ..., -0.08474077,
         -0.6558136 ,  0.8447895 ]], dtype=float32)>,
 'encoder_outputs': [<tf.Tensor: shape=(3, 128, 768), dtype=float32, numpy=
  array([[[ 0.06718684,  0.01096569, -0.16903532, ...,  0.16261348,
           -0.07787409, -0.0208286 ],
          [ 1.201476  , -0.16024688,  0.